```
conda init
conda create -n train-models python=3.10 -y
conda activate train-models
pip install torch==2.6.0+cu118 torchaudio==2.6.0+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
pip install pandas ipykernel tqdm
```

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
# Paths
DATA_DIR = Path("../VoxCeleb2")

JPG_EMB_PATH = DATA_DIR / "train/jpg_train_embeddings.pt"
JPG_VAL_EMB_PATH = DATA_DIR / "val/jpg_val_embeddings.pt"
WAV_EMB_PATH = DATA_DIR / "train/wav_train_embeddings.pt"
WAV_VAL_EMB_PATH = DATA_DIR / "val/wav_val_embeddings.pt"

In [3]:
# Load embeddings
jpg_train_emb = torch.load(JPG_EMB_PATH)
jpg_val_emb   = torch.load(JPG_VAL_EMB_PATH)

wav_train_emb = torch.load(WAV_EMB_PATH)
wav_val_emb   = torch.load(WAV_VAL_EMB_PATH)

print("Train JPG:", jpg_train_emb.shape, "Train WAV:", wav_train_emb.shape)
print("Val JPG  :", jpg_val_emb.shape, "Val WAV  :", wav_val_emb.shape)

Train JPG: torch.Size([29498, 512]) Train WAV: torch.Size([29498, 512])
Val JPG  : torch.Size([3381, 512]) Val WAV  : torch.Size([3381, 512])


In [ ]:
# Define MLP model
class FaceToSpeechMLP(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 512)
        )
        
    def forward(self, x):
        return self.model(x)

In [11]:
# Dataset / DataLoader
BATCH_SIZE = 128

train_ds = TensorDataset(jpg_train_emb, wav_train_emb)
val_ds   = TensorDataset(jpg_val_emb,   wav_val_emb)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

In [12]:
# Training Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [13]:
# Training parameters
model = FaceToSpeechMLP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

NUM_EPOCHS = 50
best_val_loss = float('inf')
MODEL_PATH = DATA_DIR / "face2speech_best.pt"

In [14]:
# Define loss function
def cosine_loss(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    pred_norm = F.normalize(pred, dim=1)
    target_norm = F.normalize(target, dim=1)
    loss = 1 - (pred_norm * target_norm).sum(dim=1).mean()
    return loss

In [15]:
# Training loop with validation
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)

        loss = cosine_loss(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            val_out = model(x_val)

            loss = cosine_loss(val_out, y_val)
            val_loss += loss.item() * x_val.size(0)
            
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.6f} - Val Loss: {val_loss:.6f}")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
        print("Saved best model")


Epoch 1/50: 100%|██████████| 231/231 [00:00<00:00, 267.38it/s]


Epoch 1/50 - Train Loss: 0.338370 - Val Loss: 0.596779
Saved best model


Epoch 2/50: 100%|██████████| 231/231 [00:00<00:00, 377.75it/s]


Epoch 2/50 - Train Loss: 0.262117 - Val Loss: 0.594339
Saved best model


Epoch 3/50: 100%|██████████| 231/231 [00:00<00:00, 379.19it/s]


...

Epoch 48/50 - Train Loss: 0.181619 - Val Loss: 0.655232


Epoch 49/50: 100%|██████████| 231/231 [00:00<00:00, 376.48it/s]


Epoch 49/50 - Train Loss: 0.181165 - Val Loss: 0.653200


Epoch 50/50: 100%|██████████| 231/231 [00:00<00:00, 366.39it/s]

Epoch 50/50 - Train Loss: 0.181008 - Val Loss: 0.655941
